<a href="https://colab.research.google.com/github/majorquev/Introduccion_a_ciencia_de_datos_R/blob/master/Clases/Clase7/Clase7_2_clasificacion_parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parte 9: Modelos de clasificación en R


**Autor:** Miguel Jorquera Viguera<br>
**E-mail:** majorquev@gmail.com<br>
**Ayudante:** Bastián Aballay Leiva<br>
**E-mail ayudante:** bastian.aballayleiva@gmail.com

## Árboles de decisión

Para la implementación de árboles de decisión (CART - classification and regression trees), utilizaremos el package `rpart` y el packages `visNetwork` (en Rstudio) para su visulaización. Otro package para visualizar árboles `rpart.plot`

## Titanic (Continuación)
Una vex realizado el paso de exploración, trataremos de predecir si una persona sobrevivirá al hundimiento del titanic utilizando árboles de clasificación.

In [ ]:
#install.packages("rpart.plot")
#install.packages("visNetwork")


In [ ]:
library(rpart) # permite generar árboles de clasificación
library(tidyverse)
library(rpart.plot)#para visualizar un árbol un poco mejor
library(visNetwork)


In [ ]:


data2 <- read.csv("https://raw.githubusercontent.com/majorquev/Introduccion_a_ciencia_de_datos_R/master/Clases/Clase7/titanic_v2.csv")
data2
glimpse(data2)



data2$Survived <- as.factor(data2$Survived)

summary(data2$Survived)


In [ ]:

#Dividimos en train y test
set.seed(112)

test <- sample_frac(data2, 0.3)
train <- anti_join(data2,test)

# guardamos en disco estos datasets para otros modelos y evaluaciones

save(test,train,file= "train_test.RData")
# load("train_test.RData")





In [ ]:
# Árbol de decisión -------------------------------------------------------

mod_arbol <- rpart(Survived ~ Pclass +
                               Sex +
                               Age +
                               SibSp +
                               Parch + 
                               Fare +
                               Embarked +
                               titulo + 
                               tipo_familia +
                               menor + 
                               madre,
          data = train
      )




In [ ]:
class(mod_arbol)
str(mod_arbol)

mod_arbol$frame
mod_arbol$where #el nodo al cual llegó cada observación de train
mod_arbol$variable.importance # a mayor valor , más importante
mod_arbol$cptable # contiene la complejidad del arbol según cantidad de nodos finales (nos va ayudar a evitar sobreajuste)




In [ ]:
# predicciones ------------------------------------------------------------

predicciones <- predict(mod_arbol,test)
prob_pred <- predicciones[,2]
predicciones <- predicciones[,2] > 0.8


mc <- table(test$Survived, predicciones)

mc

In [ ]:

# Visualización arbol -----------------------------------------------------
plot(mod_arbol)
rpart.plot(mod_arbol)





In [ ]:
# utilizando visNetwork (puede requerir dependencia del package sparkline)

visTree(mod_arbol) # En entornno de notebooks no funciona, si en Rstidio

In [ ]:

# Poda --------------------------------------------------------------------

mod_arbol_2 <- rpart(Survived ~ Pclass +
                     Sex +
                     Age +
                     SibSp +
                     Parch + 
                     Fare +
                     Embarked +
                     titulo + 
                     tipo_familia +
                     menor + 
                     madre,
                   data = train,
                   control = rpart.control(cp = 0.000001,minsplit = 2)
)
#cp : parámetro de complejidad. 

rpart.plot(mod_arbol_2)


plotcp(mod_arbol_2)
printcp(mod_arbol_2)

arbol_pod <- prune(mod_arbol_2, cp = 0.034) #permite podar el arbol

rpart.plot(arbol_pod)
## Guardo test con predicciones de arbol podado y sobreajustado
save(test,file = "test_predicciones.RData")




In [ ]:
# Evaluación como modelo predictivo ---------------------------------------

pred <- predict(arbol_pod,newdata = select(test,-Survived))
pred_or <- predict(mod_arbol_2,newdata = select(test,-Survived))

pred <- pred[,2]
pred_or <- pred_or[,2]

test$pred_arbol_pod <- pred
test$pred_arbol <- pred_or

## Asignamos categoría con un punto de corte en 0.5 
test$pred_arbol_pod_clase <- ifelse(pred > 0.5,1,0)
test$pred_arbol_clase <- ifelse(pred_or > 0.5,1,0)

mc_sobre <- table(select(test,pred_arbol_clase,Survived))
mc_pod <- table(select(test,pred_arbol_pod_clase,Survived))

